# Setup
> Monday August 14th, 2023

In [1]:
#| default_exp utils

# Previewing The Initial Datasets
> Loading Datasets using Pandas

In [2]:
#| export
from nbdev.showdoc import *
import pandas as pd
import re

In [3]:
#| export
contact_methods = pd.read_csv('data/contact_methods.csv').fillna('')
contacts = pd.read_csv('data/contacts.csv').fillna('')
gifts = pd.read_csv('data/gifts.csv').fillna('')

In [4]:
contacts.head()

,Number,Company Name,First Name,Last Name,Street,City,State,Postal,Phone,E-mail,Remarks,Deceased?
0,653377813-7,,Karita & Kelvin,Lumbers,4 Bunting Parkway,Washington,DC,20535-871,kklumbers@ yahoo.co,,Is anonymous,
1,390551098-7,,Helga,Benech,48684 Jenifer Way,Las Vegas,NV,89130,,ebenech1@goodreads.com,,
2,093004505-X,,Masha,,353 Schmedeman Park,Indianapolis,IN,,,577-374-96523,,
3,729707142-0,A Company Co.,,,2055 Lakewood Parkway,Camden,NJ,8104,,,,No
4,488464926-5,,Hoyt,Castille,37 8th Trail,Grand Rapids,MI,49560,,fcastille4@timesonline.co.uk,,No


In [5]:
gifts.head()

,donor_number,gift_id,first_name,last_name,amount received,date,fund_id,credit card type,payment method,pledge_number,notes
0,848348568-0,95196378.0,Mannie,Turpin,$4.15,3/4/2019,,,PayPal,,
1,729707142-0,95196889.0,Cymbre,Cross,2.3648,3/5/2019,ChildSponsorship,,check,,
2,687119652-8,95197689.0,Ruggiero,Makepeace,$1.31,3/7/2019,,,cash,,
3,653377813-7,95198998.0,Karita,Lumbers,$2.04,3/10/2019,,American Ex,credit card,,In honor of Mannie Turpin
4,390551098-7,95198999.0,Helga,Benech,$5.80,2019/1/10,,,cash,89752384.0,


In [6]:
contact_methods.head()

,donor_number,Phone,E-mail,Fax
0,653377813-7,832-442-4988,,
1,390551098-7,,ebenech1@goodreads.com,
2,093004505-X,818-323-9865,,818-156-7985
3,729707142-0,,,
4,488464926-5,,fcastille4@timesonline.co.uk,


Before I begin the performing the analysis in each method's respective notebook, I'm going to standardize the column names and types across the 3 DataFrames so I can pass them to each notebook and save myself some tedious typing**<br>
<br>
** Also doing this to show usefullness of juptyer notebook in production environments :)

## Housekeeping 1
> Format Column Names
<br>

- Nothing worse than malformed column names, amiright?

In [7]:
#| exports
def to_camel_case(s):
    # Remove all non-alphanumeric characters and replace with a space
    s = re.sub(r'[^a-zA-Z0-9]', ' ', s)
    
    # Split by space and capitalize the first letter of each word
    words = s.split()
    return ''.join(word.capitalize() for word in words)

In [8]:
#| exports
def transform_cnames(df, func=to_camel_case):
    df.columns = df.columns.map(func)
    return None

In [9]:
#| exports
for df in [contact_methods, contacts, gifts]:
    transform_cnames(df)

In [10]:
#| hide
contacts.columns

Index(['Number', 'CompanyName', 'FirstName', 'LastName', 'Street', 'City',
       'State', 'Postal', 'Phone', 'EMail', 'Remarks', 'Deceased'],
      dtype='object')

## Housekeeping 2
> Cleaning the column types

I identified 2 columns on the gift table that should be ints, replacing those values

In [11]:
#| exports
int_cols = ['GiftId', 'PledgeNumber']

In [12]:
#| exports
gifts[int_cols] = gifts[int_cols].replace({'':0}).astype(int)

Looks like AmountRecieved should really be a float, I'm removing any special characters (besides dashes and periods) and converting to a float :)

In [13]:
#| exports
gifts['AmountReceived'] = gifts.AmountReceived.apply(lambda x: float(re.sub(r'[^a-zA-Z0-9\.-]', '', x)))

Replacing 0 with unique id

In [14]:
gifts.head(5)

,DonorNumber,GiftId,FirstName,LastName,AmountReceived,Date,FundId,CreditCardType,PaymentMethod,PledgeNumber,Notes
0,848348568-0,95196378,Mannie,Turpin,4.1500,3/4/2019,,,PayPal,0,
1,729707142-0,95196889,Cymbre,Cross,2.3648,3/5/2019,ChildSponsorship,,check,0,
2,687119652-8,95197689,Ruggiero,Makepeace,1.3100,3/7/2019,,,cash,0,
3,653377813-7,95198998,Karita,Lumbers,2.0400,3/10/2019,,American Ex,credit card,0,In honor of Mannie Turpin
4,390551098-7,95198999,Helga,Benech,5.8000,2019/1/10,,,cash,89752384,


In [15]:
#| exports
gifts.loc[ gifts.PledgeNumber == 0, 'PledgeNumber'] = gifts[gifts.PledgeNumber == 0].index
gifts.loc[ gifts.GiftId == 0, 'GiftId'] = gifts[gifts.GiftId == 0].index

In [16]:
#| exports
gifts = gifts.rename(columns={'PledgeNumber': 'LegacyPledgeID', 'GiftId': 'LegacyGiftId'})

## Housekeeping 3
> Identify and clean any pieces of data in the wrong column

In [17]:
contacts[['Phone', 'EMail']].head(3)

,Phone,EMail
0,kklumbers@ yahoo.co,
1,,ebenech1@goodreads.com
2,,577-374-96523


Creating a function that uses regular expressions to identify a string as a phone or a email

In [18]:
#| exports
def classify_phone_email(value):
    if "@" in value:
        return "email"
    if re.search(r'\d{3}-\d{3}-\d{4}', value):
        return "phone"
    return None

Looping over phone & email records.<br>
<br>
If a phone and/or email are identified in the wrong column, they will be swapped

In [19]:
#| exports
for index, row in contacts.iterrows():
    phone_classification = classify_phone_email(row['Phone'])
    email_classification = classify_phone_email(row['EMail'])

    if phone_classification == "email":
        contacts.at[index, 'EMail'] = row['Phone']
        contacts.at[index, 'Phone'] = ''

    if email_classification == "phone":
        contacts.at[index, 'Phone'] = row['EMail']
        contacts.at[index, 'EMail'] = ''

In [20]:
contacts[['Phone', 'EMail']].head(3)

,Phone,EMail
0,,kklumbers@ yahoo.co
1,,ebenech1@goodreads.com
2,577-374-96523,


## Housekeeping 4
>Consolidating contacts table <br>
<br>
- Searching for missing users <br>
- Splitting / joining households


### Checking for Missing records <br><br>

Checking for any DonorNumbers that are NOT IN contacts Number...

In [21]:
(~gifts.DonorNumber.isin(contacts.Number.unique())).any()

True

Found 1! <br>
<br>
Doing the same for contact_methods

In [22]:
(~contact_methods.DonorNumber.isin(contacts.Number.unique())).any()

False

Didn't find any there <br>
Going to extract the missing records from the gifts table

In [23]:
#| exports
donors_not_in_contacts = gifts.loc[~gifts.DonorNumber.isin(contacts.Number.unique()), :]

In [24]:
donors_not_in_contacts

,DonorNumber,LegacyGiftId,FirstName,LastName,AmountReceived,Date,FundId,CreditCardType,PaymentMethod,LegacyPledgeID,Notes
9,809975531-Y,9,Adeline,Shakespeare,8.48,8/14/2019,,AMEX,credit card,9,
27,809975531-Y,27,Adeline,Shakespeare,7.58,8/14/2019,"Color run, ChildSponsorship",Mastercard,credit card,27,


Adding to Shakespeare to our contacts!

In [25]:
#| exports
contacts = pd.concat([
    contacts,
    # Dataframe of donors not in contacts
    pd.DataFrame(donors_not_in_contacts[['DonorNumber', 'FirstName', 'LastName']]
                 .drop_duplicates()
                 .rename(columns={'DonorNumber': 'Number'})
                 .drop_duplicates()
                 .to_dict('records'))
])

### Spliting rows with Multiple People

In [26]:
contacts.FirstName.head(1)

0    Karita & Kelvin
Name: FirstName, dtype: object

Split the names on ' & ' or ' and ', then expand the resulting lists into new rows

In [27]:
#| exports
contacts[['FirstName', 'SecondaryFirstName']] = contacts['FirstName'].str.split(' & | and ', expand=True).fillna('')

Aditionally, going to look for any records where there's a duplicated `Number` <br>
<br>
I'm assuming `Number` is a primary key for a household or organization

In [28]:
#| exports
records_to_join = contacts.loc[contacts.Number.duplicated(), :].to_dict(orient='records')

 Removing duplicates from contacts before joining

In [29]:
#| exports
contacts = contacts.loc[~contacts.Number.duplicated(), :]

Adding dupliates as secondary contacts

In [30]:
#| exports
for record in records_to_join:
    contacts.loc[contacts.Number.isin([record['Number']]), ['SecondaryFirstName', 'SecondaryLastName']] = [record['FirstName'], record['LastName']]

In [31]:
#| export
contacts['SecondaryLastName'] = contacts.SecondaryLastName.fillna('')

Adding Secondary Last Name for the appropriate users

In [32]:
#| exports
contacts['SecondaryLastName'] = contacts.apply(lambda x: x['LastName'] if x['SecondaryLastName'] == '' and x['SecondaryFirstName'] != '' else x['SecondaryLastName'], axis=1)

Initializing new columns

In [33]:
#| export
contacts[['LegacyIndividualId', 'SecondaryLegacyIndividualId']] = None

In [34]:
#| export
contacts.reset_index(inplace=True, drop=True)

Adding id since none was provided

In [35]:
#| exports
id = 0
for index, row in contacts.iterrows():
    contacts.loc[index, 'LegacyIndividualId'] = id
    id += 1
    if row['SecondaryFirstName'] != '':
        contacts.loc[index, 'SecondaryLegacyIndividualId'] = id
        id += 1


In [36]:
#| export
contacts.fillna('', inplace=True)

## Housekeeping 5
>Cleaning Records with blank first or last names

Checking for records where FirstName  and/or LastName is blank


In [37]:
#| exports
blank_name_records = ((contacts.FirstName == '') | (contacts.LastName == ''))

Previewing the blank name records

In [38]:
contacts.loc[((contacts.FirstName == '') | (contacts.LastName == '')), :]

,Number,CompanyName,FirstName,LastName,Street,City,State,Postal,Phone,EMail,Remarks,Deceased,SecondaryFirstName,SecondaryLastName,LegacyIndividualId,SecondaryLegacyIndividualId
2,093004505-X,,Masha,,353 Schmedeman Park,Indianapolis,IN,,577-374-96523,,,,,,3,
3,729707142-0,A Company Co.,,,2055 Lakewood Parkway,Camden,NJ,8104,,,,No,,,4,
7,029456846-8,,,,608 Old Shore Alley,Marietta,GA,30066,,jdoley6@telegraph.co.uk,,,,,9,


Before I delete the records I'm going to check if the names are present on the gift table <br>
<br>
I'm going to start by getting the unique Numbers that the records belong too

In [39]:
#| exports
blank_name_numbers= contacts.loc[blank_name_records, 'Number']

In [40]:
gifts.loc[gifts.DonorNumber.isin(blank_name_numbers), ['DonorNumber', 'FirstName', 'LastName']].drop_duplicates().head(5)

,DonorNumber,FirstName,LastName
1,729707142-0,Cymbre,Cross
6,029456846-8,Romy,Doley
7,093004505-X,Masha,Butt Gow
19,729707142-0,Cymbre,


The names are present on the gifts table!

In [41]:
#| exports
gift_name_records = gifts.loc[gifts.DonorNumber.isin(blank_name_numbers), ['DonorNumber', 'FirstName', 'LastName']].drop_duplicates()

Removing the invalid record

In [42]:
#| exports
gift_name_records = gift_name_records.loc[((gift_name_records.FirstName != '') & (gift_name_records.LastName != '')), :]
gift_name_records

,DonorNumber,FirstName,LastName
1,729707142-0,Cymbre,Cross
6,029456846-8,Romy,Doley
7,093004505-X,Masha,Butt Gow


Updating the records that previousuly had a blank first or last name

In [43]:
#| exports
for _, row in gift_name_records.iterrows():
    contacts.loc[contacts['Number'] == row['DonorNumber'], ['FirstName', 'LastName']] = [row['FirstName'], row['LastName']]

All the records valid names now!

In [44]:
contacts.loc[blank_name_records, :]

,Number,CompanyName,FirstName,LastName,Street,City,State,Postal,Phone,EMail,Remarks,Deceased,SecondaryFirstName,SecondaryLastName,LegacyIndividualId,SecondaryLegacyIndividualId
2,093004505-X,,Masha,Butt Gow,353 Schmedeman Park,Indianapolis,IN,,577-374-96523,,,,,,3,
3,729707142-0,A Company Co.,Cymbre,Cross,2055 Lakewood Parkway,Camden,NJ,8104,,,,No,,,4,
7,029456846-8,,Romy,Doley,608 Old Shore Alley,Marietta,GA,30066,,jdoley6@telegraph.co.uk,,,,,9,


## Housekeeping 6
>Adding Contact Name

In [45]:
contacts.head(3)

,Number,CompanyName,FirstName,LastName,Street,City,State,Postal,Phone,EMail,Remarks,Deceased,SecondaryFirstName,SecondaryLastName,LegacyIndividualId,SecondaryLegacyIndividualId
0,653377813-7,,Karita,Lumbers,4 Bunting Parkway,Washington,DC,20535-871,,kklumbers@ yahoo.co,Is anonymous,,Kelvin,Lumbers,0,1
1,390551098-7,,Helga,Benech,48684 Jenifer Way,Las Vegas,NV,89130,,ebenech1@goodreads.com,,,,,2,
2,093004505-X,,Masha,Butt Gow,353 Schmedeman Park,Indianapolis,IN,,577-374-96523,,,,,,3,


Creating a function to update the contact name depending on the individuals on the household record

In [46]:
#| exports
def set_contact_name(row):
    if row['LastName'] == row['SecondaryLastName']:
        return row['FirstName'] + ' & ' + row['SecondaryFirstName'] + ' ' + row['LastName']
    elif row['SecondaryFirstName'] != '':
        return row['FirstName'] +  ' ' + row['LastName'] + ' & ' + row['SecondaryFirstName'] + ' ' + row['SecondaryLastName']
    else:
        return row['FirstName'] + ' ' + row['LastName']

In [47]:
#| exports
contacts['ContactName'] = contacts.apply(set_contact_name, axis=1)

## Housekeeping 7
>Splitting Project Codes

In [48]:
#| exports
project_codes = gifts.FundId.str.split(', ', expand=True)

In [49]:
project_codes.head(3)

,0,1
0,,None
1,ChildSponsorship,None
2,,None


In [50]:
#| exports
gifts[['Project1Code', 'Project2Code']] = project_codes

In [51]:
#| exports
gifts = gifts.loc[:, gifts.columns.drop('FundId')].copy()

## Housekeeping 8
>Formating date

In [52]:
df['Date'].head(5)

0     3/4/2019
1     3/5/2019
2     3/7/2019
3    3/10/2019
4    2019/1/10
Name: Date, dtype: object

Since Dates are in different formats I'm going to create a custom parser

In [53]:
#| export
from datetime import datetime

In [54]:
#| exports
def custom_parser(date_str):
    try:
        return datetime.strptime(date_str, '%m/%d/%Y')
    except ValueError:
        return datetime.strptime(date_str, '%Y/%m/%d')

Applying parser

In [55]:
#| exports
gifts['GiftDate'] = gifts['Date'].apply(custom_parser)

Dropping old date columns

In [56]:
#| exports
gifts = gifts.loc[:, gifts.columns.drop('Date')].copy()

## Housekeeping 9
>Validate emails

Previewing the `Email` column

In [57]:
contacts.EMail.head(1)

0    kklumbers@ yahoo.co
Name: EMail, dtype: object

It looks like the first email 'kklumbers@ yahoo.co' is invalid and should be fixed <br>
<br>
I'm going to write a function to validate emails and fix common mistakes

In [58]:
#| exports
def valid_email(s):
    pattern = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
    if pattern.match(s):
        return True
    return False

In [59]:
#| exports
def fix_email(email):
    if not email:
        return ''
    
    email = re.sub(r'\s', '', email)

    # Check and fix common mistakes in top-level domain (TLD)
    for wrong_tld, correct_tld in [('co', 'com'), ('cmo', 'com'), ('con', 'com')]:
        if email.endswith('.' + wrong_tld):
            email = email[:-len(wrong_tld)] + correct_tld

    # Check and fix common mistakes in domain names
    for wrong_domain, correct_domain in [('gmial', 'gmail'), ('yahho', 'yahoo')]:
        email = email.replace('@' + wrong_domain, '@' + correct_domain)

    if valid_email(email):
        return email

    return ''


Testing the email validator

In [60]:
contacts.EMail.apply(fix_email)

0              kklumbers@yahoo.com
1           ebenech1@goodreads.com
2                                 
3                                 
4     fcastille4@timesonline.co.uk
5                                 
6                                 
7          jdoley6@telegraph.co.uk
8             cmakepeace7@1688.com
9                                 
10                                
Name: EMail, dtype: object

Saving the change

In [61]:
#| exports
contacts['EMail'] = contacts.EMail.apply(fix_email)

## Housekeeping 10
>Fix phone numbers

There appears to be a invalid phone number

In [62]:
contacts.loc[2, 'Phone']

'577-374-96523'

Writing a function to validate US phone numbers, assuming the phone numbers provided are from the US

In [63]:
#| exports
def validate_us_phone_number(phone_number):
    # Patterns for different US phone number formats
    patterns = [
        r'^\+1\s?\d{3}-\d{3}-\d{4}$',
        r'^\(\d{3}\)\s?\d{3}-\d{4}$',
        r'^\d{3}-\d{3}-\d{4}$',
        r'^\d{3}-\d{4}$' 
    ]

    # Check if the phone number matches any of the patterns
    for pattern in patterns:
        if re.match(pattern, phone_number):
            return phone_number

    return ''

In [64]:
#| exports
contacts['Phone'] = contacts.Phone.apply(validate_us_phone_number)

In [65]:
#| exports
contacts.fillna('', inplace=True)

# Export

In [66]:
#| hide
import nbdev

In [67]:
#| hide
nbdev.nbdev_export('00_Setup.ipynb')

<br>